## Shared Bikes Demand Prediction 

#### Problem Statement:

A US bike-sharing provider `BoomBikes` has a daily dataset on the rental bikes based on various environmental and seasonal settings. It wishes to use this data to understand the factors affecting the demand for these shared bikes in the American market and come up with a mindful business plan to be able to accelerate its revenue as soon as the ongoing lockdown due to corona pandemic comes to an end.

Essentially, the company wants to know —


- Which variables are significant in predicting the demand for shared bikes.


- How well those variables describe the bike demands


The solution is divided into the following sections: 
- Data understanding and exploration
- Data Visualisation 
- Data preparation
- Model building and evaluation


### 1. Data Understanding and Exploration

Let's first import the required libraries and have a look at the dataset and understand the size, attribute names etc.

In [1032]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

In [1033]:
import warnings
warnings.filterwarnings('ignore')

In [1034]:
# Reading the dataset
bike_sharing = pd.read_csv("day.csv")

In [1035]:
# Let's take a look at the first few rows
bike_sharing.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [1036]:
# Let's look at the number of rows and columns in the dataset
bike_sharing.shape

(730, 16)

In [1037]:
# Understanding the feature names in the dataset
bike_sharing.columns

Index(['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'casual', 'registered', 'cnt'],
      dtype='object')

In [1038]:
# Getting insights of the features
bike_sharing.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,365.500000,2.498630,0.500000,6.526027,0.028767,2.997260,0.683562,1.394521,20.319259,23.726322,62.765175,12.763620,849.249315,3658.757534,4508.006849
std,210.877136,1.110184,0.500343,3.450215,0.167266,2.006161,0.465405,0.544807,7.506729,8.150308,14.237589,5.195841,686.479875,1559.758728,1936.011647
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.424346,3.953480,0.000000,1.500244,2.000000,20.000000,22.000000
25%,183.250000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,13.811885,16.889713,52.000000,9.041650,316.250000,2502.250000,3169.750000
50%,365.500000,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,20.465826,24.368225,62.625000,12.125325,717.000000,3664.500000,4548.500000
75%,547.750000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,26.880615,30.445775,72.989575,15.625589,1096.500000,4783.250000,5966.000000
max,730.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,35.328347,42.044800,97.250000,34.000021,3410.000000,6946.000000,8714.000000


In [1039]:
# Summary of the dataset: 730 rows, 16 columns, no null values
print(bike_sharing.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    int64  
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(11), object(1)
memory usage: 91.4+ KB
None


#### Understanding the Data Dictionary and parts of Data Preparation

The data dictionary contains the meaning of various attributes; some of which are explored and manipulated here:

In [1040]:
bike_sharing

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,27-12-2019,1,1,12,0,4,1,2,10.420847,11.33210,65.2917,23.458911,247,1867,2114
726,727,28-12-2019,1,1,12,0,5,1,2,10.386653,12.75230,59.0000,10.416557,644,2451,3095
727,728,29-12-2019,1,1,12,0,6,0,2,10.386653,12.12000,75.2917,8.333661,159,1182,1341
728,729,30-12-2019,1,1,12,0,0,0,1,10.489153,11.58500,48.3333,23.500518,364,1432,1796


In [1041]:
# Assigning string values to different seasons instead of numeric values.
season_mapping = {1: 'spring', 2: 'summer', 3: 'fall', 4: 'winter'}
bike_sharing['season'] = bike_sharing['season'].map(season_mapping)

In [1042]:
bike_sharing.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [1043]:
# Checking whether the conversion is done properly or not and getting data count on the basis of season
bike_sharing['season'].astype('category').value_counts()

fall      188
summer    184
spring    180
winter    178
Name: season, dtype: int64

In [1044]:
# year (0: 2018, 1:2019)
bike_sharing['yr'].astype('category').value_counts()

0    365
1    365
Name: yr, dtype: int64

In [1045]:
# Assigning string values to different months instead of numeric values which may misindicate some order to it.
# A function has been created to map the actual numbers to categorical levels.
def object_map(x):
    return x.map({1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul',8: 'Aug',9: 'Sept',10: 'Oct',11: 'Nov',12: 'Dec'})

# Applying the function to the two columns
bike_sharing[['mnth']] = bike_sharing[['mnth']].apply(object_map)

In [1046]:
bike_sharing

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,Jan,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,Jan,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,Jan,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,Jan,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,Jan,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,27-12-2019,spring,1,Dec,0,4,1,2,10.420847,11.33210,65.2917,23.458911,247,1867,2114
726,727,28-12-2019,spring,1,Dec,0,5,1,2,10.386653,12.75230,59.0000,10.416557,644,2451,3095
727,728,29-12-2019,spring,1,Dec,0,6,0,2,10.386653,12.12000,75.2917,8.333661,159,1182,1341
728,729,30-12-2019,spring,1,Dec,0,0,0,1,10.489153,11.58500,48.3333,23.500518,364,1432,1796


In [1047]:
# Checking whether the conversion is done properly or not and getting data count on the basis of month
bike_sharing['mnth'].astype('category').value_counts()

Aug     62
Dec     62
Jan     62
Jul     62
Mar     62
May     62
Oct     62
Apr     60
Jun     60
Nov     60
Sept    60
Feb     56
Name: mnth, dtype: int64

In [1048]:
# whether day is a holiday or not (0: No, 1: Yes)
bike_sharing['holiday'].astype('category').value_counts()

0    709
1     21
Name: holiday, dtype: int64

In [1049]:
bike_sharing

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,Jan,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,Jan,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,Jan,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,Jan,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,Jan,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,726,27-12-2019,spring,1,Dec,0,4,1,2,10.420847,11.33210,65.2917,23.458911,247,1867,2114
726,727,28-12-2019,spring,1,Dec,0,5,1,2,10.386653,12.75230,59.0000,10.416557,644,2451,3095
727,728,29-12-2019,spring,1,Dec,0,6,0,2,10.386653,12.12000,75.2917,8.333661,159,1182,1341
728,729,30-12-2019,spring,1,Dec,0,0,0,1,10.489153,11.58500,48.3333,23.500518,364,1432,1796


In [1050]:
# Assigning string values to weekdays instead of numeric values. These values may misindicate some order to it.
# A function has been created to map the actual numbers to categorical levels.
def str_map(x):
    return x.map({1: 'Wed', 2: 'Thurs', 3: 'Fri', 4: 'Sat', 5: 'Sun', 6: 'Mon', 0: 'Tues'})

# Applying the function to the two columns
bike_sharing[['weekday']] = bike_sharing[['weekday']].apply(str_map)

In [1051]:
# Checking whether the conversion is done properly or not and getting data count on the basis of weekdays
bike_sharing['weekday'].astype('category').value_counts()

Mon      105
Tues     105
Wed      105
Sat      104
Sun      104
Thurs    104
Fri      103
Name: weekday, dtype: int64

In [1052]:
# if a day is neither weekend nor a holiday it takes the value 1, otherwise 0
bike_sharing['workingday'].astype('category').value_counts()

1    499
0    231
Name: workingday, dtype: int64

In [1053]:
# Replacing long weathersit names into string values for better readability and understanding
weathersit_mapping = {
    1: 'A',
    2: 'B',
    3: 'C',
    4: 'D'
}
bike_sharing['weathersit'] = bike_sharing['weathersit'].map(weathersit_mapping)

In [1054]:
# Extracting the type of weather situations present in the data
bike_sharing['weathersit'].unique()

array(['B', 'A', 'C'], dtype=object)

In [1055]:
# Taking count based on weather situations
bike_sharing['weathersit'].astype('category').value_counts()

A    463
B    246
C     21
Name: weathersit, dtype: int64

In [1056]:
bike_sharing.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,Jan,0,Mon,0,B,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,Jan,0,Tues,0,B,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,Jan,0,Wed,1,A,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,Jan,0,Thurs,1,A,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,Jan,0,Fri,1,A,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [ ]:
#Auto EDA
!pip install sweetviz
import sweetviz as sv
sweet_report = sv.analyze(bike_sharing)
sweet_report.show_html('sweet_report.html')

### 2. Data Visualisation

Let's now spend some time doing what is arguably the most important step - **understanding the data**.
- Understanding the distribution of various numeric variables 
- If there is some obvious multicollinearity going on, this is the first place to catch it
- Here's where you'll also identify if some predictors directly have a strong association with the outcome variable

We'll visualise our data using `matplotlib` and `seaborn`.

In [ ]:
# temperature
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(8, 10))

# Plot temperature histogram on the first subplot
sns.histplot(x=bike_sharing['temp'], kde=True, ax=ax1)
ax1.set_xlabel('Temperature (normalized)')
ax1.set_ylabel('Frequency')
ax1.set_title('Distribution of Temperature')

# Plot temperature density plot on the second subplot
sns.kdeplot(x=bike_sharing['temp'], shade=True, ax=ax2)
ax2.set_xlabel('Temperature (normalized)')
ax2.set_ylabel('Density')
ax2.set_title('Density Plot of Temperature')

plt.tight_layout()
plt.show()


In [ ]:
# feeling temperature
# Temperature and Feeling Temperature histogram
fig, axs = plt.subplots(1, 2, figsize=(10,5))
sns.distplot(bike_sharing['temp'], ax=axs[0])
sns.distplot(bike_sharing['atemp'], ax=axs[1])
axs[0].set(title='Temperature Distribution', xlabel='Temperature')
axs[1].set(title='Feeling Temperature Distribution', xlabel='Feeling Temperature')

# Mean Feeling Temperature line
mean_feeling_temp = bike_sharing['atemp'].mean()
axs[1].axvline(x=mean_feeling_temp, color='r', linestyle='--')
axs[1].text(mean_feeling_temp, 0.02, f'Mean Feeling Temp: {mean_feeling_temp:.2f}', rotation=90, va='bottom', ha='right', transform=axs[1].get_xaxis_transform())

plt.tight_layout()
plt.show()


In [ ]:
# humidity
sns.histplot(bike_sharing['hum'], kde=True, stat='density')
plt.xlabel('Humidity')
plt.ylabel('Density')
plt.title('Distribution of Humidity')
plt.show()


In [ ]:
# wind speed
sns.histplot(bike_sharing['windspeed'], kde=True, stat='density')
plt.xlabel('Wind Speed')
plt.ylabel('Density')
plt.title('Distribution of Wind Speed')
plt.show()

In [ ]:
# Target variable: count of total rental bikes including both casual and registered
sns.histplot(bike_sharing['cnt'], kde=True, stat='density')
plt.xlabel('Count of Total Rental Bikes')
plt.ylabel('Density')
plt.title('Distribution of Total Rental Bikes')
plt.show()

In [ ]:
# Converting date to datetime format
bike_sharing['dteday']=bike_sharing['dteday'].astype('datetime64')

In [ ]:
# All categorical variables in the dataset
bike_sharing_categorical=bike_sharing.select_dtypes(exclude=['float64','datetime64','int64'])
print(bike_sharing_categorical.columns)

In [ ]:
bike_sharing_categorical

#### Visualising Categorical Variables

As you might have noticed, there are a few categorical variables as well. Let's make a boxplot for some of these variables.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

sns.lineplot(x='dteday', y='cnt', hue='yr', style='mnth', data=bike_sharing, ax=ax)

ax.set_xlabel('Date')
ax.set_ylabel('Total rentals')
ax.set_title('Total Bike Rentals by Year and Month')

plt.show()


In [ ]:

plt.figure(figsize=(20, 20))  
plt.subplot(3,3,1)
sns.boxplot(x = 'season', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,2)
sns.boxplot(x = 'holiday', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,3)
sns.boxplot(x = 'workingday', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,4)
sns.boxplot(x = 'weathersit', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,5)
sns.boxplot(x = 'mnth', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,6)
sns.boxplot(x = 'weekday', y = 'cnt', data = bike_sharing)
plt.subplot(3,3,7)
sns.boxplot(x = 'yr', y = 'cnt', data = bike_sharing)
plt.show()

#### Visualising Numeric Variables

Let's make a pairplot of all the numeric variables

In [ ]:
# Converting "casual","registered" and "cnt" numeric variables to float. 
# This step is performed to seperate out categorical variables like 'yr','holiday','workingday' which have binary values in them
IntVariableList = ["casual","registered","cnt"]

for var in IntVariableList:
    bike_sharing[var] = bike_sharing[var].astype("float")

In [ ]:
# All numeric variables in the dataset
bike_sharing_numeric = bike_sharing.select_dtypes(include=['float64'])
bike_sharing_numeric.head()

In [ ]:
# Pairwise scatter plot
sns.pairplot(bike_sharing_numeric)
plt.show()

We can better plot correlation matrix between variables to know the exact values of correlation between them. Also, a heatmap is pretty useful to visualise multiple correlations in one plot.

In [ ]:
# Correlation matrix
cor = bike_sharing_numeric.corr()
cor

Let's plot the correlations on a heatmap for better visualisation

In [ ]:
# heatmap
mask = np.array(cor)
mask[np.tril_indices_from(mask)] = False
fig,ax= plt.subplots()
fig.set_size_inches(10,10)
sns.heatmap(cor, mask=mask,vmax=.8, square=True,annot=True)

The heatmap shows some useful insights:

Correlation of Count('cnt') with independent variables:
- Count('cnt') is highly (positively) correlated with 'casual' and 'registered' and further it is high with 'atemp'. We can clearly understand the high positive correlation of count with 'registered' and 'casual' as both of them together add up to represent count.

- Count is negatively correlated to 'windspeed' (-0.24 approximately). This gives us an impression that the shared bikes demand will be somewhat less on windy days as compared to normal days.

Correlation among independent variables:
- Some of the independent variables are highly correlated (look at the top-left part of matrix): atemp and temp are highly (positively) correlated. The correlation between the two is almost equal to 1.


Thus, while building the model, we'll have to pay attention to multicollinearity.

In [ ]:
#removing atemp as it is highly correlated with temp
bike_sharing.drop('atemp',axis=1,inplace=True)    

## 3. Data Preparation 


#### Data Preparation

Let's now prepare the data and build the model.
Note that we had not included 'yr', 'mnth', 'holiday', 'weekday' and 'workingday' as object variables in the initial data exploration steps so as to avoid too many dummy variables creation. They have binary values: 0s and 1s in them which have specific meanings associated with them.

In [ ]:
# Subset all categorical variables
bike_sharing_categorical=bike_sharing.select_dtypes(include=['object'])

#### Dummy Variables
The variable `season`,`mnth`,`weekday` and `weathersit` have different levels. We need to convert these levels into integers. 

For this, we will use something called `dummy variables`.

In [ ]:
# Convert into dummies
bike_sharing_dummies = pd.get_dummies(bike_sharing_categorical, drop_first=True)
bike_sharing_dummies.head()

In [ ]:
# Drop categorical variable columns
bike_sharing = bike_sharing.drop(list(bike_sharing_categorical.columns), axis=1)

In [ ]:
# Concatenate dummy variables with the original dataframe
bike_sharing = pd.concat([bike_sharing, bike_sharing_dummies], axis=1)

In [ ]:
# Let's check the first few rows
bike_sharing.head()

In [ ]:
# Drop the 'instant' and 'dteday' column as they of not any use to us for the analysis
bike_sharing=bike_sharing.drop(['instant','dteday'], axis = 1, inplace = False)
bike_sharing.head()

## 4. Model Building and Evaluation

Let's start building the model. The first step to model building is the usual test-train split. So let's perform that

In [ ]:
# Split the dataframe into train and test sets
from sklearn.model_selection import train_test_split
np.random.seed(0)
df_train, df_test = train_test_split(bike_sharing, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
df_train

### Scaling

Now that we have done the test-train split, we need to scale the variables for better interpretability. But we only need the scale the numeric columns and not the dummy variables. Let's take a look at the list of numeric variables we had created in the beginning. Also, the scaling has to be done only on the train dataset as you don't want it to learn anything from the test data.

Let's scale all these columns using MinMaxScaler. You can use any other scaling method as well; it is totally up to you.

In [ ]:
from sklearn.preprocessing import MinMaxScaler 

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
var = ['temp', 'hum', 'windspeed','casual','registered','cnt']

df_train[var] = scaler.fit_transform(df_train[var])

In [ ]:
df_train

As expected, the variables have been appropriately scaled.

In [ ]:
df_train.describe()

In [ ]:
# Let's check the correlation coefficients to see which variables are highly correlated
plt.figure(figsize = (30, 30))
sns.heatmap(df_train.corr(), annot = True, cmap="YlGnBu")
plt.show()

As you might have noticed, `temp` seems to the correlated to `cnt` the most, after 'casual' and 'registered'. Let's see a pairplot for `temp` vs `cnt`.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(df_train.temp, df_train.cnt, alpha=0.5)
ax.set_xlabel('Temperature (C)', fontsize=16)
ax.set_ylabel('Number of Rentals', fontsize=16)
ax.set_title('Temperature vs. Rentals', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.show()

#### Dividing into X and Y sets for the model building

In [ ]:
# Dropping 'casual' and 'registered' as together they add up to cnt
y_train = df_train.pop('cnt')
X_train = df_train.drop(["casual","registered"],axis=1) 

In [ ]:
X_train.head()

In [ ]:
# This is done to convert all the features into array before fitting the model and avoid any error popping up
np.asarray(df_train)

In [ ]:
X_train.shape

### Building the first model with all the features

Let's now build our first model with all the features.

In [ ]:
import statsmodels.api as sm
X_train_lm = sm.add_constant(X_train)

lr = sm.OLS(y_train, X_train_lm).fit()

lr.params

In [ ]:
# Instantiate
lm = LinearRegression()

# Fit a line
lm.fit(X_train, y_train)

In [ ]:
# Print the coefficients and intercept
print(lm.coef_)
print(lm.intercept_)

In [ ]:
# getting the model summary
lr.summary()

This model has an Adjusted R-squared value of **84.5%** which seems pretty good. But let's see if we can reduce the number of features and exclude those which are not much relevant in explaining the target variable. 

#### Model Building Using RFE

Now, you have close to 28 features. It is obviously not recommended to manually eliminate these features. So let's now build a model using recursive feature elimination to select features. We'll first start off with an arbitrary number of features (15 seems to be a good number to begin with), and then use the `statsmodels` library to build models using the shortlisted features (this is also because `SKLearn` doesn't have `Adjusted R-squared` that `statsmodels` has).

In [ ]:
# Import RFE
from sklearn.feature_selection import RFE

# RFE with 15 features
lm = LinearRegression()
rfe1 = RFE(lm, n_features_to_select=15,step=1)

# Fit with 15 features
rfe1.fit(X_train, y_train)

# Print the boolean results
print(rfe1.support_)           
print(rfe1.ranking_)  

#### Model Building and Evaluation 

Let's now check the summary of this model using `statsmodels`.

In [ ]:
# Import statsmodels
import statsmodels.api as sm  

# Subset the features selected by rfe1
col1 = X_train.columns[rfe1.support_]

# Subsetting training data for 15 selected columns
X_train_rfe1 = X_train[col1]

# Add a constant to the model
X_train_rfe1 = sm.add_constant(X_train_rfe1)
X_train_rfe1.head()

In [ ]:
# Fitting the model with 15 variables
lm1 = sm.OLS(y_train, X_train_rfe1).fit()   
print(lm1.summary())

Note that the new model built on the selected features doesn't show much dip in the accuracy in comparison to the model which was built on all the features. It has gone from **84.5%** to **84.4%**. This is indeed a good indication to proceed with these selected features.

But let's check for the multicollinearity among these variables.

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
a=X_train_rfe1.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = a.columns
vif['VIF'] = [variance_inflation_factor(a.values, i) for i in range(a.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

In [ ]:
# Import RFE
from sklearn.feature_selection import RFE

# RFE with 7 features
lm = LinearRegression()
rfe2 = RFE(lm, n_features_to_select=7)

# Fit with 7 features
rfe2.fit(X_train, y_train)

# Print the boolean results
print(rfe2.support_)           
print(rfe2.ranking_)  

In [ ]:
# Import statsmodels
import statsmodels.api as sm  

# Subset the features selected by rfe1
col1 = X_train.columns[rfe2.support_]

# Subsetting training data for 7 selected columns
X_train_rfe2 = X_train[col1]

# Add a constant to the model
X_train_rfe2 = sm.add_constant(X_train_rfe2)
X_train_rfe2.head()

In [ ]:
# Fitting the model with 7 variables
lm2 = sm.OLS(y_train, X_train_rfe2).fit()   
print(lm2.summary())

Now let's check the VIF for these selected features and decide further.

In [ ]:
b=X_train_rfe2.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = b.columns
vif['VIF'] = [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

From the model summary above, all the variables have p-value < 0.05 and from the p-value perspective, all variables seem significant. But notice that there are a few variables which have VIF > 5. We need to deal with these variables carefully.

So let's try removing 'hum' first having the maximum VIF and then check for it again. Dropping this variable may result in a change in other VIFs which are high.

In [ ]:
# Let's drop the 'hum' column
X_train_rfe2.drop("hum",axis=1,inplace=True)
X_train_rfe2

In [ ]:
X_train_rfe2 = sm.add_constant(X_train_rfe2)

# Now that we have removed one variable, let's fit the model with 6 variables
lm3 = sm.OLS(y_train, X_train_rfe2).fit()   
print(lm3.summary())

The model seems to be doing a good job. Let's also quickly take a look at the VIF values.

In [ ]:
c=X_train_rfe2.drop('const',axis=1)

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs except for the constant
vif = pd.DataFrame()
vif['Features'] = c.columns
vif['VIF'] = [variance_inflation_factor(c.values, i) for i in range(c.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

All the VIF values and p-values seem to be in the permissible range now. Also the `Adjusted R-squared` value has dropped from `84.5%` with **28 variables** to just `79.1%` using **6 variables**. This model is explaining most of the variance without being too complex. So let's proceed with this model.

### Residual Analysis

Before we make predictions on the test set, let's first analyse the residuals.

In [ ]:
y_train_cnt = lm3.predict(X_train_rfe2)

In [ ]:
# Plot the histogram of the error terms
fig, ax = plt.subplots(figsize=(8,6))
sns.histplot((y_train - y_train_cnt), bins=20, ax=ax, kde=True, color='steelblue', alpha=0.8)

# Set plot title and axis labels
ax.set_title('Distribution of Error Terms', fontsize=20)
ax.set_xlabel('Error', fontsize=16)
ax.set_ylabel('Frequency', fontsize=16)

# Set tick label sizes
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=12)

# Set plot background color and gridlines
ax.set_facecolor('#f9f9f9')
ax.grid(axis='y', linestyle='--', alpha=0.7)

# Add a vertical line at zero error
ax.axvline(x=0, color='r', linestyle='--', lw=2)

# Add legend
ax.legend(['Zero Error', 'Error Distribution'], fontsize=14)


The error terms are fairly normally distributed and we can surely live with this. Let's now make predictions on the test-set.

### Making Predictions

We would first need to scale the test set as well. So let's start with that.

In [ ]:
X_train_rfe2

In [ ]:
# let's recall the set of variables which are to be scaled
var

In [ ]:
df_test[var] = scaler.transform(df_test[var])

In [ ]:
# Split the 'df_test' set into X and y after scaling
y_test = df_test.pop('cnt')
X_test = df_test.drop(["casual","registered"],axis=1)

In [ ]:
X_test.head()

In [ ]:
# Let's check the list 'col2' which had the 6 variables RFE had selected
col2=c.columns
col2

In [ ]:
# Let's subset these columns and create a new dataframe 'X_test_rfe1'
X_test_rfe2 = X_test[col2]

In [ ]:
# Add a constant to the test set created
X_test_rfe2 = sm.add_constant(X_test_rfe2)
X_test_rfe2.info()

In [ ]:
# Making predictions
y_pred = lm3.predict(X_test_rfe2)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(y_test, y_pred, alpha=0.5)
ax.plot(ax.get_xlim(), ax.get_ylim(), ls="--", c=".3")
ax.set_xlabel('Actual values', fontsize=16)
ax.set_ylabel('Predicted values', fontsize=16)
ax.set_title('Actual vs. Predicted Values', fontsize=20)
plt.show()


From the above plot, it's evident that the model is doing well on the test set as well. Let's also check the R-squared and more importantly, the adjusted R-squared value for the test set.

In [ ]:
# r2_score for 6 variables
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

Thus, for the model with 6 variables, the r-squared on training and test data is about 79.3% and 78.02% respectively. The adjusted r-squared on the train set is about is about 79.1%.

#### Checking the correlations between the final predictor variables

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(bike_sharing[col2].corr(), cmap="YlGnBu", annot=True, ax=ax)
ax.set_title("Correlation Matrix", fontsize=20)
ax.set_xlabel("Features", fontsize=16)
ax.set_ylabel("Features", fontsize=16)
ax.tick_params(axis='both', which='major', labelsize=12)
plt.show()

This is the simplest model that we could build. The final predictors seem to have fairly low correlations. 

Thus, the final model consists of the 6 variables mentioned above.One can go ahead with this model and use it for predicting count of daily bike rentals.

